In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Load the base model
model_path = r'C:\Users\RDITLADI\Documents\AMPL\code\all_run_dir\drag_run_dir\results_drag_coefficient\saved_models\Cd_drag_estimate_top_0_model.keras'
base_model = tf.keras.models.load_model(model_path)

# Load dataset
data_path = r'c:\Users\RDITLADI\Documents\AMPL\code\all_run_dir\drag_run_dir\drag_data\sc1095_30AUG2023_noZeros.drag.csv'
data = pd.read_csv(data_path)

In [2]:
# Separate features and target variable
X = data[['Temp', 'LWC', 'Mach_env', 'AoA_env', 'Mach_c81', 'AoA_c81']]
y = data['Cd']

In [3]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [4]:
# Convert to numpy arrays for model compatibility
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [5]:
# Number of models for the Boosting ensemble
n_boosting_models = 10

In [6]:
# Initialize predictions
train_predictions = np.zeros(len(y_train))
test_predictions = np.zeros(len(y_test))

In [7]:
for i in range(n_boosting_models):
    # Calculate residuals
    residuals = y_train - train_predictions

In [9]:
    # Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# Train the model on the residuals
model.fit(X_train, residuals, epochs=50, verbose=0)

In [15]:
    # Update the predictions
train_predictions += model.predict(X_train).flatten()
test_predictions += model.predict(X_test).flatten()

3669/3669 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
408/408 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step


In [16]:
# Evaluate the ensemble
mse = mean_squared_error(y_test, test_predictions)
print(f'Ensemble MSE: {mse}')

Ensemble MSE: 0.36235909478706213
